<font face="Roboto" size=4>
<div dir=rtl align=center>
<br>
<img src="https://aut.ac.ir/templates/tmpl_modern01/images/logo_fa.png" alt="Amirkabir University Logo" width="100">
<br>
<font size=6>
<b>Implementing Pac-Man Game Using Minimax Algorithm</b>
<br>
<b><font size=5>Artificial Intelligence Course</b>
<br>
<b><font size=5>Professor: Dr. Mehdi Ghatee</b>
<br>
<font size=4>
<b>Mahsa Goodarzi - 9912043</b>
<br>
<hr>
</div>
</font>

# IMPORTS

In [26]:
from random import choice
from datetime import datetime
from math import sqrt
from time import sleep

# DEFINITION OF THE GAME ENVIRONMENT

In [27]:
LOGO = '''
'  ██████   █████   ██████       ███    ███  █████  ███    ██
'  ██   ██ ██   ██ ██            ████  ████ ██   ██ ████   ██
'  ██████  ███████ ██      █████ ██ ████ ██ ███████ ██ ██  ██
'  ██      ██   ██ ██            ██  ██  ██ ██   ██ ██  ██ ██
'  ██      ██   ██  ██████       ██      ██ ██   ██ ██   ████
'
'''

PACMAN_START_POSITION = (8, 0)
GHOST1_START_POSITION = (5, 6)
GHOST2_START_POSITION = (12, 6)

OBSTACLES = ([(4, 0), (13, 0), (1, 1), (2, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (13, 1),
              (15, 1), (16, 1), (1, 2), (16, 2), (1, 3), (3, 3), (4, 3), (13, 3), (14, 3), (16, 3), (6, 4), (7, 4),
              (8, 4), (9, 4), (10, 4), (11, 4), (1, 5), (3, 5), (4, 5), (13, 5), (14, 5), (16, 5), (1, 6), (16, 6),
              (1, 7), (2, 7), (4, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (13, 7), (15, 7), (16, 7),
              (4, 8), (13, 8)] + [(-1, y) for y in range(-1, 10)] + [(18, y) for y in range(-1, 10)] +
             [(x, -1) for x in range(0, 18)] + [(x, 9) for x in range(0, 18)])

food = [(x, y) for x in range(0, 18) for y in range(0, 9) if (x, y) not in OBSTACLES]
food.remove(PACMAN_START_POSITION)

# DEFINITION OF e_utility FUNCTION AND RELATED FUNCTIONS

In [28]:
def e_utility(pacman_position: tuple, ghost1_position: tuple, ghost2_position: tuple, food_position: list) -> int:
    score = 0

    # If Pacman is hit by a ghost, 500 points will be deducted from the score
    if pacman_position == ghost1_position or pacman_position == ghost2_position:
        score -= 500

    # If Pacman eats a food point, 20 points will be added to the score
    if pacman_position in food_position:
        score += 20

    # For each valid move for Pacman, 5 points are added to the score
    score += len(get_valid_moves(pacman_position)) * 5

    # -20 * distance value points are deducted from score (distance: the distance between Pacman and the closest food)
    score += -20 * the_closest_food(pacman_position, food_position)

    # 10 * distance value points are deducted from score (distance: the distance between Pacman and the closest ghost)
    score += 10 * the_closest_ghost(pacman_position, [ghost1_position, ghost2_position])

    # For each ghost in the vicinity of Pacman position 15 points are deducted from the score
    score += -15 * number_of_nearby_ghosts(pacman_position, [ghost1_position, ghost2_position])

    # For each Pacman move on the screen, 1 point is deducted from the score
    score -= 1

    return score


def the_closest_food(pacman_position: tuple, food_position: list) -> int:
    """
    This function calculates the distance between Pacman and the closest food and returns this distance value.
    :param pacman_position:
    :param food_position:
    :return: distance: The distance from the closest food
    """
    distance = min([sqrt(
        abs(pacman_position[0] - food_point_position[0]) ** 2 + abs(pacman_position[1] - food_point_position[1]) ** 2)
        for food_point_position in food_position])
    return int(distance)


def the_closest_ghost(pacman_position: tuple, ghosts_position: list) -> int:
    """
    This function calculates the distance between Pacman and the closest ghost and returns this distance value.
    :param pacman_position:
    :param ghosts_position:
    :return: distance: The distance from the closest ghost
    """
    distance = min([sqrt(abs(pacman_position[0] - ghost[0]) ** 2 + abs(pacman_position[1] - ghost[1]) ** 2) for ghost in
                    ghosts_position])
    return int(distance)


def number_of_nearby_ghosts(pacman_position: tuple, ghosts_position: list) -> int:
    """
    This function returns the number of ghosts that are adjacent to Pacman.
    :param pacman_position:
    :param ghosts_position:
    :return: number_of_ghosts: The number of ghosts
    """
    number_of_ghosts = 0
    for ghost in ghosts_position:
        if ghost in get_valid_moves(pacman_position):
            number_of_ghosts += 1
    return number_of_ghosts

# MINIMAX ALGORITHM

In [29]:
def value(pacman_position: tuple, ghost1_position: tuple, ghost2_position: tuple, depth: int, next_agent: str) -> float:
    """
    This function determines which of min_value or max_value functions will be returned as output according to the
    next node, or if the depth of the search tree reaches its end or the game is in its terminal state,
    e_utility function will be returned. If the next node (next_agent) is Pacman, max_value function should be returned,
    and if the next node is ghost 1 or ghost 2, min_value function should be returned.
    :param pacman_position:
    :param ghost1_position:
    :param ghost2_position:
    :param depth:
    :param next_agent:
    :return: e_utility function or max_value function or min_value function
    """
    global food
    if depth == 0 or end_game(pacman_position, ghost1_position, ghost2_position, food) != 0:
        return e_utility(pacman_position, ghost1_position, ghost2_position, food)
    if next_agent == "PacMan":
        return max_value(pacman_position, ghost1_position, ghost2_position, depth)
    elif next_agent == "Ghost1" or next_agent == "Ghost2":
        return min_value(pacman_position, ghost1_position, ghost2_position, depth, current_agent=next_agent)


def max_value(pacman_position: tuple, ghost1_position: tuple, ghost2_position: tuple, depth: int) -> float:
    """
    This function finds the successors of Pacman node, and with the help of a for loop, returns the successor with
    the highest value as output.
    :param pacman_position:
    :param ghost1_position:
    :param ghost2_position:
    :param depth:
    :return: maximum_value: The highest value of the successors of Pacman
    """
    successors = get_valid_moves(pacman_position)
    maximum_value = float('-inf')

    for successor in successors:
        new_pacman_position = successor
        score = value(new_pacman_position, ghost1_position, ghost2_position, depth=depth - 1, next_agent="Ghost1")
        maximum_value = max(maximum_value, score)

    return maximum_value


def min_value(pacman_position: tuple, ghost1_position: tuple, ghost2_position: tuple, depth: int,
              current_agent: str) -> float:
    """
    This function first determines whether the current node is ghost1 or ghost2, and if it is either of these two,
    it finds the successors of this node and with the help of a for loop, it finds the node with the lowest value for
    Pacman.
    :param pacman_position:
    :param ghost1_position:
    :param ghost2_position:
    :param depth:
    :param current_agent:
    :return: minimum_value: The lowest value of successors of a ghost (ghost1 or ghost2)
    """
    minimum_value = float('inf')

    if current_agent == "Ghost1":
        successors = get_valid_moves(ghost1_position)

        for successor in successors:
            new_ghost1_position = successor
            score = value(pacman_position, new_ghost1_position, ghost2_position, depth=depth - 1, next_agent="Ghost2")
            minimum_value = min(minimum_value, score)

    elif current_agent == "Ghost2":
        successors = get_valid_moves(ghost2_position)

        for successor in successors:
            new_ghost2_position = successor
            score = value(pacman_position, ghost1_position, new_ghost2_position, depth=depth - 1, next_agent="PacMan")
            minimum_value = min(minimum_value, score)

    return minimum_value

# TERMINAL STATE

In [30]:
def end_game(pacman_position: tuple, ghost1_position: tuple, ghost2_position: tuple, current_food: list) -> int:
    """
    This function assigns a number to each of the game terminal states of the game and returns it as an output.
    :param pacman_position:
    :param ghost1_position:
    :param ghost2_position:
    :param current_food:
    :return: 1 if Pacman eats all the food points, -1 if one of the ghosts eats Pacman, 0 if none of these occur
    """
    if pacman_position == ghost1_position or pacman_position == ghost2_position:
        return -1
    if len(current_food) == 0:
        return 1
    return 0

# GET VALID NEXT POSITIONS

In [31]:
def get_valid_moves(agent_position: tuple) -> list:
    """
    After receiving the current position of the agent, this function returns a list of valid positions that the agent
    can move to.
    :param agent_position:
    :return: valid_moves: valid positions that the agent can move to
    """
    x, y = agent_position
    valid_moves = []

    # Move Up
    if (x, y + 1) not in OBSTACLES:
        valid_moves.append((x, y + 1))
    # Move Down
    if (x, y - 1) not in OBSTACLES:
        valid_moves.append((x, y - 1))
    # Move Right
    if (x + 1, y) not in OBSTACLES:
        valid_moves.append((x + 1, y))
    # Move Left
    if (x - 1, y) not in OBSTACLES:
        valid_moves.append((x - 1, y))

    # Agent Staying In Place
    valid_moves.append(agent_position)

    return valid_moves

# DISPLAY OF THE GAME

In [32]:
def display_game_screen(pacman_position: tuple, ghost1_position: tuple, ghost2_position: tuple, food_position: list):
    """
    This function displays the game screen at each level with the help of a for loop. Pacman is represented by 🟨,
    spirits by 🟦, food by 🟢, obstacles by 🟫, and empty spaces by ⚪.
    :param pacman_position:
    :param ghost1_position:
    :param ghost2_position:
    :param food_position:
    :return: None
    """
    for j in range(9, -2, -1):
        line = ""
        for i in range(-1, 19):
            if (i, j) in OBSTACLES:
                line += " 🟫 "
            elif (i, j) == ghost1_position or (i, j) == ghost2_position:
                line += " 🟦 "
            elif (i, j) == pacman_position:
                line += " 🟨 "
            elif (i, j) in food_position:
                line += " 🟢 "
            else:
                line += " ⚪ "
        print(line)
    print()


# PLAY GAME FUNCTION

In [33]:
def play_game(depth, call_sleep_fun):
    start_time = datetime.now()
    global food

    # The Basic Environment of The Game
    print(LOGO)
    print()
    print(f"Position Of Obstacles: {OBSTACLES}")
    print(f"Position Of Food Points: {food}")
    print()
    print(f"Pacman Starts From: {PACMAN_START_POSITION}")
    print(f"Ghost-1 Starts From: {GHOST1_START_POSITION}")
    print(f"Ghost-2 Starts From: {GHOST2_START_POSITION}")
    print()

    pacman_position = PACMAN_START_POSITION
    ghost1_position = GHOST1_START_POSITION
    ghost2_position = GHOST2_START_POSITION
    current_score = 0
    level = 0

    while end_game(pacman_position, ghost1_position, ghost2_position, food) == 0:

        # See the display of the game screen in each level at a slower speed
        if call_sleep_fun:
            sleep(1)

        print(f"Level: {level}")
        print(f"Current Score: {current_score}")
        print(f"Number Of Food Points Remaining: {len(food)}")
        print(f"Current Pacman Position: {pacman_position}")
        print(f"Current Ghost1 Position: {ghost1_position}")
        print(f"Current Ghost2 Position: {ghost2_position}")
        print()
        display_game_screen(pacman_position, ghost1_position, ghost2_position, food)
        print()

        # Determine The Depth
        depth = depth
        # Starting The Minimax Algorithm
        # Pacman is the root and node of Max
        successors = get_valid_moves(pacman_position)
        best_move = None
        maximum_value = float('-inf')

        for successor in successors:
            new_pacman_position = successor
            score = value(new_pacman_position, ghost1_position, ghost2_position, depth - 1, next_agent="Ghost1")
            # Choosing The Most Beneficial Move For Pacman
            if score > maximum_value:
                maximum_value = score
                best_move = new_pacman_position

        pacman_position = best_move

        # For each Pacman move on the screen, 1 point is deducted from the score
        current_score -= 1
        # If Pacman eats a food point, 20 points will be added to the score
        if pacman_position in food:
            food.remove(pacman_position)
            current_score += 10

        # Random Movement of Ghost1 and Ghost2
        ghost1_position = choice(get_valid_moves(ghost1_position))
        ghost2_position = choice(get_valid_moves(ghost2_position))

        # Win or Lose
        if end_game(pacman_position, ghost1_position, ghost2_position, food) == 1:
            print("<<<<<----- 🏆 Pacman Won! All Food Points Were Eaten! 🏆 ----->>>>>")
        elif end_game(pacman_position, ghost1_position, ghost2_position, food) == -1:
            print("<<<<<----- 👾 Oops! Pacman Was Eaten By A Ghost! 👾 ----->>>>>")

        level += 1

    # Calculation of Execution Game Time
    end_time = datetime.now()
    execution_time = end_time - start_time

    print(f"Final Pacman Position: {pacman_position}")
    print(f"Final Ghost1 Position: {ghost1_position}")
    print(f"Final Ghost2 Position: {ghost2_position}")
    print(f"Final Score: {current_score}")
    print(f"Number Of Levels: {level}")
    print(f"Execution Time: {execution_time}")
    display_game_screen(pacman_position, ghost1_position, ghost2_position, food)

# RUN GAME

In [34]:
play_game(depth=4, call_sleep_fun=False)


'  ██████   █████   ██████       ███    ███  █████  ███    ██
'  ██   ██ ██   ██ ██            ████  ████ ██   ██ ████   ██
'  ██████  ███████ ██      █████ ██ ████ ██ ███████ ██ ██  ██
'  ██      ██   ██ ██            ██  ██  ██ ██   ██ ██  ██ ██
'  ██      ██   ██  ██████       ██      ██ ██   ██ ██   ████
'


Position Of Obstacles: [(4, 0), (13, 0), (1, 1), (2, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (13, 1), (15, 1), (16, 1), (1, 2), (16, 2), (1, 3), (3, 3), (4, 3), (13, 3), (14, 3), (16, 3), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 4), (1, 5), (3, 5), (4, 5), (13, 5), (14, 5), (16, 5), (1, 6), (16, 6), (1, 7), (2, 7), (4, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (13, 7), (15, 7), (16, 7), (4, 8), (13, 8), (-1, -1), (-1, 0), (-1, 1), (-1, 2), (-1, 3), (-1, 4), (-1, 5), (-1, 6), (-1, 7), (-1, 8), (-1, 9), (18, -1), (18, 0), (18, 1), (18, 2), (18, 3), (18, 4), (18, 5), (18, 6), (18, 7), (18, 8), (18, 9), (0, -1), (1, -1), (2, -1), (3, -1), (4,